In [34]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("train.csv") 
df_train.head()

dix = pd.DatetimeIndex(df_train.datetime)
df_train['hour']=pd.DatetimeIndex(df_train.datetime).hour
df_train['day']=pd.DatetimeIndex(df_train.datetime).dayofweek
df_train['month']=pd.DatetimeIndex(df_train.datetime).month

df_train_origin = df_train   #保存原数据
df_train = df_train.drop(['datetime','casual','registered','windspeed'], axis = 1)  #丢弃不要的字段

#['season','holiday','workingday','weather','temp','atemp', 'humidity','windspeed','count','month','day','hour']

df_train_target = df_train['count'].values
df_train_data = df_train.drop(['count'],axis = 1).values

In [35]:
# linear_model
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

from sklearn import linear_model
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn import svm

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)  # 切分数据

In [36]:
print("岭回归")    
for train, test in kf.split(df_train_data):    
    ridgeModel = linear_model.Ridge().fit(df_train_data[train], df_train_target[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        ridgeModel.score(df_train_data[train], df_train_target[train]), 
        ridgeModel.score(df_train_data[test], df_train_target[test])))

岭回归
train score: 0.340, test score: 0.325

train score: 0.335, test score: 0.347

train score: 0.337, test score: 0.338

train score: 0.341, test score: 0.324

train score: 0.335, test score: 0.345



In [38]:
print("支持向量回归/SVR(kernel='rbf',C=10,gamma=.001)")
for train, test in kf.split(df_train_data):
    
    svmModel = svm.SVR(kernel ='rbf', C = 10, gamma = .001).fit(df_train_data[train], df_train_target[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        svmModel.score(df_train_data[train], df_train_target[train]), 
        svmModel.score(df_train_data[test], df_train_target[test])))

支持向量回归/SVR(kernel='rbf',C=10,gamma=.001)
train score: 0.425, test score: 0.385

train score: 0.421, test score: 0.403

train score: 0.417, test score: 0.420

train score: 0.408, test score: 0.448

train score: 0.415, test score: 0.421



In [33]:
print("回归树/DecisionTreeRegressor(criterion = 'mse')")    
for train, test in kf.split(df_train_data):
    dtrModel = RandomForestRegressor(criterion = 'mse').fit(df_train_data[train], df_train_target[train])
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        dtrModel.score(df_train_data[train], df_train_target[train]), 
        dtrModel.score(df_train_data[test], df_train_target[test])))

回归树/DecisionTreeRegressor(criterion = 'mse')


NameError: name 'RandomForestRegressor' is not defined

In [8]:
X = df_train_data
y = df_train_target

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=0)

#,'linear','poly'
tuned_parameters = [{'kernel':['rbf','linear']}]   
    
scores = ['r2']

for score in scores:
    
    print(score)
    
    grid = GridSearchCV(svm.SVR(), tuned_parameters, cv=5, scoring=score)
    grid.fit(X_train, y_train)

    print('网格搜索-最佳度量值:',grid.best_score_)  # 获取最佳度量值
    print('网格搜索-最佳参数：',grid.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
    print('网格搜索-最佳模型：',grid.best_estimator_)  # 获取最佳度量时的分类器模型
    print("")

r2
网格搜索-最佳度量值: 0.27083492137848814
网格搜索-最佳参数： {'kernel': 'linear'}
网格搜索-最佳模型： SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

